In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/My\ Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'   dcgan_mnist   gan_mnist   img.zip


In [0]:
### GAN (Generative Adversarial Networks)
### for mnist

### import
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np

### GAN class
class GAN():
  def __init__(self):
    self.img_rows = 28
    self.img_cols = 28
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self. channels)

    self.z_dim = 100

    optimizer = Adam(0.0002, 0.5)

    ## discriminator
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss = 'binary_crossentropy',
                              optimizer = optimizer,
                              metrics = ['accuracy'])

    ## generator
    self.generator = self.build_generator()

    ## combined
    self.combined = self.build_combined()
    self.combined.compile(loss = 'binary_crossentropy',
                         optimizer = optimizer)


  def build_discriminator(self):
    img_shape = self.img_shape

    model = Sequential()

    model.add(Flatten(input_shape = img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dense(1, activation = 'sigmoid'))

    model.summary()

    return model

  def build_generator(self):
    noise_shape = (self.z_dim,)

    model = Sequential()

    model.add(Dense(256, input_shape = noise_shape))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Dense(np.prod(self.img_shape), activation = 'tanh'))
    model.add(Reshape(self.img_shape))

    model.summary()

    return model

  def build_combined(self):
    self.discriminator.trainable = False
    model = Sequential([self.generator, self.discriminator])
    return model

  def train(self, epochs, batch_size=128, save_interval=50):
    (x_train, _), (_, _) = mnist.load_data()

    x_train = (x_train.astype(np.float32) - 127.5) / 127.5
    x_train = np.expand_dims(x_train, axis = 3)

    half_batch = int(batch_size / 2)

    for epoch in range(epochs):

      ## train discriminator 
      noise = np.random.normal(0, 1, (half_batch, self.z_dim))
      gen_imgs = self.generator.predict(noise)

      idx = np.random.randint(0, x_train.shape[0], half_batch)
      imgs = x_train[idx]

      d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      ## train generator
      noise = np.random.normal(0, 1, (batch_size, self.z_dim))
      valid_y = np.array([1] * batch_size)
      g_loss = self.combined.train_on_batch(noise, valid_y)

      print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]"%(epoch, d_loss[0], 100*d_loss[1], g_loss))

      if epoch % save_interval == 0:
        self.save_imgs(epoch)

  def save_imgs(self, epoch):
    r, c = 5, 5
    
    noise = np.random.normal(0, 1, (r*c, self.z_dim))
    gen_imgs = self.generator.predict(noise)
    
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
      for j in range(c):
        axs[i, j].imshow(gen_imgs[cnt, :,:,0], cmap = 'gray')
        axs[i, j].axis('off')
        cnt += 1
    fig.savefig("/content/drive/My Drive/gan_mnist/mnist_%d.png" % epoch)
    plt.close()

if __name__ == '__main__':
  gan = GAN()
  gan.train(epochs = 30000, batch_size=32, save_interval=500)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.111557, acc.: 15.62%] [G loss: 0.612575]
1 [D loss: 0.453139, acc.: 81.25%] [G loss: 0.654218]
2 [D loss: 0.305143, acc.: 90.62%] [G loss: 0.788783]
3 [D loss: 0.389370, acc.: 68.75%] [G loss: 0.921929]
4 [D loss: 0.227404, acc.: 93.75%] [G loss: 1.014471]
5 [D loss: 0.238355, acc.: 93.75%] [G loss: 1.109867]
6 [D loss: 0.214002, acc.: 100.00%] [G loss: 1.193600]
7 [D loss: 0.216638, acc.: 93.75%] [G loss: 1.340113]
8 [D loss: 0.171774, acc.: 100.00%] [G loss: 1.402920]
9 [D loss: 0.185029, acc.: 100.00%] [G loss: 1.498397]
10 [D loss: 0.143787, acc.: 100.00%] [G loss: 1.559411]
11 [D loss: 0.127306, acc.: 100.00%] [G loss: 1.744322]
12 [D loss: 0.099902, acc.: 100.00%] [G loss: 1.785557]
13 [D loss: 0.126161, acc.: 100.00%] [G loss: 1.883677]
14 [D loss: 0.109919, acc.: 100.00%] [G loss: 1.881077]
15 [D loss: 0.111232, acc.: 100.00%] [G loss: 1.915071]
16 [D loss: 0.102237, acc.: 100.00%] [G loss: 2.072420]
17 [D loss: 0.067823, acc.: 100.00%] [G loss: 2.086535]
18 [D los